# Spectra acquisiton
Control both the stepping motor and the camera from this notebook.<br>
Make a loop to measure a wide-range spectrum automatically.

NOTE: only one connection to COM port is possible at a time. Shut down other connections, either in python or dedicated software.

In [25]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [26]:
import time, os, logging
from FLI import FLI
import numpy as np
import xarray as xr
from os.path import join

In [27]:
%run thr640.py
%run imaging_tool.py

In [29]:
logger = logger
logger.setLevel(50) # 10 - DEBUG 50 - CRITICAL 20 - INFO
camera = FLI()
step_motor = THR640() # Can only have one instance.

In [36]:
basepath = r'\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027'
os.path.exists(basepath)

True

In [37]:
join(basepath,'a.nc')

'\\\\10.249.254.51\\Public\\takeoka\\Spectraacquisition_takeoka_3_20201027\\a.nc'

In [38]:
exposure = 1.5 * 1e2 # ms
vbin = 1
attributes = {
    "temperature": camera.getTemperature(),
    "device_status": camera.getDeviceStatus(),
    "exposure": exposure,
    "frame_type": "light",
    "motor_coordinate":"",
}

#R = [100000 + i*5000 for i in range(101)]
#R = [605000 + i*5000 for i in range(20)]
#R = [380000 + i*5000 for i in range(25)]
#R = [500000 + i*5000 for i in range(5)]
#R = [400000 + i*9000 for i in range(15)]
R = [430000 + i*5000 for i in range(10)]
for r in log_progress(R):
    step_motor.goto(r)
    step_motor.waitUntilReady()
    attributes['motor_coordinate'] = r
    
    camera.setExposureTime(exposure)
    camera.setVBin(vbin)
    camera.setImageArea(10,0,2058,512//vbin)
    time.sleep(.5)
    camera.exposeFrame()
    time.sleep(.1)
    # load image
    image_data = camera.grabFrame(out=np.empty((512//vbin,2048), np.uint16))
    # convert to xarray
    data = xr.DataArray(
        image_data, dims=["y", "x"], coords={"image_counter": 0}, attrs=attributes
    )
    # save data as NetCDF file
    filepath = join(basepath,f'H_{r}.nc')
    print(filepath)
    data.to_netcdf(filepath)
print('Done.')

\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_430000.nc
\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_435000.nc
\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_440000.nc
\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_445000.nc
\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_450000.nc
\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_455000.nc
\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_460000.nc
\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_465000.nc
\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_470000.nc
\\10.249.254.51\Public\takeoka\Spectraacquisition_takeoka_3_20201027\H_475000.nc
Done.


In [8]:
[430000 + i*5000 for i in range(10)]

[430000,
 435000,
 440000,
 445000,
 450000,
 455000,
 460000,
 465000,
 470000,
 475000]